In [1]:
from Retrieval.retrieval import SparseRetrieval, DenseRetrieval,HybridRetrieval
from transformers import AutoTokenizer
from datasets import load_from_disk
import pickle

## Retrieval 클래스
- tokenizer, data_path, context_path를 인자로 가집니다.
- tokenizer -> 사용할 tokenizer를 사용합니다. DenseRetrieval의 경우 학습에 사용한 tokenizer를 사용해야합니다.
- context_path -> 가져올 corpus의 path를 인자로 넘겨주어야 합니다.
- data_path -> 가져온 문서나 embedding을 저장할 위치(미정)

# Retrieval 클래스의 method
- get_topk_doc_id_and_score : query에 대해서 top k개의 유사도가 높은 wiki id와 score를 가지고 옵니다.
- get_topk_doc_id_and_score_for_querys : query list 대해서 top k개의 유사도가 높은 wiki id와 score를 query - id list, query - score list 의 dict 형태로 전달해 줍니다.

In [2]:
valid_dataset = load_from_disk('/opt/ml/data/test_dataset/validation').to_pandas()
querys = valid_dataset['question']

## SparseRetrieval(elastic search) 예제

In [3]:
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base') # 토크나이저
elastic = SparseRetrieval(tokenizer=tokenizer) 
# 엘라스틱 서치는 nori 토크나이저를 사용하지만 exception이 생길 경우 bert tokenizer를 사용하는 
# BM25Okapi를 사용합니다.

Token indices sequence length is longer than the specified maximum sequence length for this model (1131 > 512). Running this sequence through the model will result in indexing errors
/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [4]:
# get_topk_doc_id_and_score를 호출하면 해당 query에 대해 유사도가 높은 wiki 문서의 id와 그 점수를 top k 개 만큼 가져옵니다.
doc_ids, scores = elastic.get_topk_doc_id_and_score(querys[0],5)
print(doc_ids)
print(scores)

[43280, 47081, 35064, 24024, 42242]
[20.487246, 20.10372, 19.851677, 19.529999, 18.778242]


In [5]:
# context를 확인 하고 싶으면 클래스 변수인 wiki_id_context_dict을 사용하면 됩니다.
print(f'question : {querys[0]}')
for i in range(0,3):
    print(f'top_{i+1} passage score : {scores[i]}')
    print(elastic.wiki_id_context_dict[doc_ids[i]])
    print('-'*100)

question : 유령'은 어느 행성에서 지구로 왔는가?
top_1 passage score : 20.487246
목성의 대기에서 보이는 줄무늬는 적도와 평행하면서 행성을 둘러싸는 대(zone)와 띠(belt)라고 불리는 물질의 반대 순환류에 의한 것이다. 대는 밝은 줄무늬로, 대기에서 상대적으로 고도가 높은 곳에 있다. 이들은 내부의 상승 기류를 가지고 있는 고기압 영역이다. 띠는 어두운 줄무늬로, 대기에서 상대적으로 고도가 낮은 곳에 있으며, 내부의 하강 기류를 가진다. 이들은 저기압 영역이다. 이러한 구조는 지구 대기의 고기압 및 저기압 세포와 어느정도 유사하나, 국지 작은 기압 세포와 상반되는 행성 전체를 둘러싸는 위도 줄무늬로서 매우 다른 구조를 가지고 있다. 이는 행성의 빠른 자전과 근본적인 대칭으로 인한 결과로 보인다. 행성에는 국지적인 가열을 일으키는 바다나 육지가 없으며 자전 속도는 지구보다 훨씬 빠르다. 행성에는 서로 다른 크기와 색상을 갖는 점과 같은 작은 구조들이 있다. 목성에서, 그러한 특색 중에서 가장 유명한 것은 대적점으로, 적어도 300년 동안 존재해 왔다. 이러한 구조의 실체는 거대한 폭풍이다. 그러한 점 중에 일부는 적란운이기도 하다.
----------------------------------------------------------------------------------------------------
top_2 passage score : 20.10372
갈리프레이 (Gallifrey)는 영국의 SF 텔레비전 드라마 《닥터 후》에서 등장하는 행성이다. 드라마의 주인공인 닥터와 마스터를 비롯한 지금까지 등장한 모든 타임 로드의 고향이다. 카스터보로스 성단 내에서 "은하 중심에서 은하좌표로 10-0-11-0-0 하고도 0-2 지점"에 위치해 있으며 쌍성계를 이루고 있다. 닥터의 고향 행성은 드라마 방영 초반에는 밝혀지지 않다가, 2대 닥터 에피소드인 The War Games (1969)에서 닥터의 고향 행성이 처음으

In [6]:
# get_topk_doc_id_and_score_for_querys를 호출하고 인자로 query가 담긴 리스트와 top k를 인자로 넘겨주면
# query와 높은 유사도를 가진 wiki id 리스트와 score 리스트를 dict로 전달해 줍니다.
q_ids, q_scores = elastic.get_topk_doc_id_and_score_for_querys(querys.to_list(),10)

print(type(q_ids))
print(type(q_scores))
print(q_ids[querys[1]])
print(q_scores[querys[1]])

100%|██████████| 600/600 [00:14<00:00, 42.81it/s]

<class 'dict'>
<class 'dict'>
[20081, 23179, 15556, 13590, 18470, 29191, 50337, 12365, 50525, 39045]
[23.784315, 21.420332, 21.085941, 20.766212, 19.601896, 19.360214, 19.141119, 18.332466, 18.24624, 18.194616]


In [7]:
print(f'question : {querys[1]}')
for i in range(0,3):
    print(f'top_{i+1} passage score : {q_scores[querys[1]][i]}')
    print(elastic.wiki_id_context_dict[q_ids[querys[1]][i]])
    print('-'*100)

question : 용병회사의 경기가 좋아진 것은 무엇이 끝난 이후부터인가?
top_1 passage score : 23.784315
SK에서 방출된 이후 그는 일본을 여행하며 잠시 신변을 정리하고 있었다. 그 때 좌완 선발 투수가 필요하여 니코스키에게 관심을 두고 있었던 두산 베어스가 SK 와이번스에 계약 양도를 신청하여 곧바로 맷 왓슨의 대체 용병으로 두산 베어스로 이적했고 두산 베어스의 선발진에 합류하여 비로소 첫 승을 따 냈으며 승리를 대부분 한화 이글스전에서 기록하여 한화 이글스에는 강력한 천적으로 자리매김했다. 그러나 예전부터 주로 원포인트 및 중간계투로 뛰다가 선발로 전환한 탓에 선발로는 아직 익숙하지 않아 3회까지 투구수가 70개 가까이 되는 경기가 대부분이라는 단점이 있었다. 2009년 9월 13일 KIA 타이거즈전에서 비로소 초반에 급격하게 늘어나는 투구수를 극복하여 승리를 따 내기도 했다. 두산 베어스 합류 후 12경기에 더 나서고 2009년 정규 시즌을 4승 8패, 평균 자책 3.78로 마무리했다. 완전한 선발 전업 후 후반기에 조금씩 나아지는 모습을 보였으나, 잠실야구장에서 롯데 자이언츠와 맞붙었던 2009년 9월 29일 준 플레이오프 1차전에서 무실점으로 3이닝을 호투하던 중 갑작스러운 어깨 통증으로 조기 강판당했고, 그 날 두산 베어스는 롯데 자이언츠에게 1차전을 내주었다. 진단 결과 극상근 손상 판정을 받아 포스트 시즌 전력에서 완전히 이탈하고 말았다. 플레이오프가 끝난 이후 이닝 소화 능력이 떨어진다는 이유로 두산 베어스가 재계약을 포기하면서 대한민국을 떠나게 되었다. 두산에서의 마지막이 좋지 않았지만, 그가 두산에 입단하여 2군에서 불펜 피칭을 했을 때 그를 지켜보았던 박종훈 당시 2군 감독은 니코스키를 두고 성격이 좋았다고 하였다.
----------------------------------------------------------------------------------------------------
top_2 pas

## DenseRetrieval(Bert) 예제

In [8]:
# query encoder, context encoder를 학습시키고 사용한 모델의 tokenizer와 모델의 저장 위치를 인자로 가집니다.
dense_retrieval = DenseRetrieval(tokenizer, 'p_encoder/','q_encoder/')

In [9]:
# 문서를 가져오는 function은 Retrieval과 동일합니다. (SparseRetrieval 참고)
doc_ids, scores = dense_retrieval.get_topk_doc_id_and_score(querys[0],10)
print(doc_ids)
print(scores)

[9727, 9257, 49262, 9615, 8988, 29581, 29064, 9671, 50628, 11628]
[319.8592529296875, 319.40545654296875, 319.20556640625, 318.966552734375, 318.74884033203125, 318.7379150390625, 318.4833068847656, 318.36279296875, 318.1867980957031, 318.1792907714844]


In [10]:
print(f'question : {querys[0]}')
for i in range(0,3):
    print(f'top_{i+1} passage score : {scores[i]}')
    print(dense_retrieval.wiki_id_context_dict[doc_ids[i]])
    print('-'*100)

question : 유령'은 어느 행성에서 지구로 왔는가?
top_1 passage score : 319.8592529296875
그라브 지역 (Graves)은 보르도 남부, 가론 강 좌안에 길게 위치하고 다음과 같은 AOC로 이뤄져 있다. * 페사크레오냥 (Pessac-Léognan AOC)은 보르도 외곽 지역근처에 위치한다. ::유명와인 : 샤토 오브리옹 (Chateau haut-brion, 1등급), 샤토 라 미시옹 오브리옹 (chateau la mission haut-brion), 샤토 파프 클르망 (chateau pape clement), 샤토 오브리앙 블랑 (chateau haut-brion blanc, 백) * 세롱 (Cérons AOC)은 그라브 지역 남부에 위치한 작은 지역이다. ::유명와인 : 샤토 갈랑 세롱 (Chateau gallant cerons, 백) * 그라브 (Graves AOC) 와 그라브 쉬페리외르 (Graves supérieurs AOC)는 나머지 그라브 지역에 사용한다.
----------------------------------------------------------------------------------------------------
top_2 passage score : 319.40545654296875
숲 여러 나무가 군락을 이루어 자라는 곳을 숲이라 한다. 사람의 손길이 닿지 않은 숲은 원시림이라 불리며 개마고원과 같은 깊은 오지에 분포한다. 도심의 숲은 생물서식처로서의 기능을 갖고 있어 비오토프라 불린다. 한대 지방의 툰드라나 열대 지방의 열대우림과 같이 기후에 따라 숲을 이루는 나무의 종류가 다르다. 기후대에 따라 주종을 이루는 나무를 극상종이라 한다. 한국의 극상종은 서어나무, 떡갈나무 등이다. 한국에서는 온도 조건에 따라 난대림, 온대림, 한대림으로 구분한다. * 난대림 : 북위 35도 이남 지역, 연평균 기온이 14℃ 이상, 상록 활엽수림 * 온대림 : 북위 35~43도(고산 지역 제외), 연평균

In [11]:
q_ids, q_scores = dense_retrieval.get_topk_doc_id_and_score_for_querys(querys.to_list(),10)

print(type(q_ids))
print(type(q_scores))
print(q_ids[querys[1]])
print(q_scores[querys[1]])

100%|██████████| 600/600 [00:00<00:00, 1687.23it/s]

<class 'dict'>
<class 'dict'>
[14238, 16310, 53846, 29851, 13555, 51707, 38386, 35983, 50310, 27693]
[332.8433837890625, 332.751220703125, 332.5208435058594, 332.357177734375, 332.3047180175781, 332.29180908203125, 332.10577392578125, 332.10296630859375, 332.01019287109375, 331.9410400390625]


In [12]:
print(f'question : {querys[1]}')
for i in range(0,3):
    print(f'top_{i+1} passage score : {q_scores[querys[1]][i]}')
    print(dense_retrieval.wiki_id_context_dict[q_ids[querys[1]][i]])
    print('-'*100)

question : 용병회사의 경기가 좋아진 것은 무엇이 끝난 이후부터인가?
top_1 passage score : 332.8433837890625
백련교의 교리는 현세의 고난으로부터 구원을 추구하는 일종의 메시아 사상으로 볼 수 있다. 창세주인 무생노모(無生老母)가 미륵을 이 세상으로 보내서 자신의 흩어진 자녀들을 거두어들여 '진공가향'(眞空家鄕)이라는 일종의 천국에 귀의시키고 평화로운 천년왕국이 인간세계에 실현될 것이라고 선전했다. 백련교도들은 불교식으로 향을 피우고 불공을 드리는 한편 새로 올 왕국을 대비하며 무술 수련도 하였다. 백련교는 중국 역사에서 두번 큰 역할을 하는데 첫 번째는 14세기 원나라가 망하고 명나라가 세워질 때 이른바 홍건적의 난의 사상적 기원이 되었다. 명의 태조 주원장도 백련교도로 출발하여 중국을 통일하고 명을 세웠다. 두 번째는 18세기 말 청나라에서 일어난 무장반란인데 이 때도 이민족인 만주족의 지배에 대항하여 한족을 중심으로 저항하는 세력으로 청나라의 쇠퇴에 큰 영향을 미쳤다. 이후에도 백련교도는 여러차례 청나라에 반대하는 비밀결사로 조직되었고 나중에 의화단의 모태가 되기도 하였다.
----------------------------------------------------------------------------------------------------
top_2 passage score : 332.751220703125
부건은 부홍(苻洪)의 셋째 아들로 317년에 태어났다. 후조(後趙)의 석호(石虎)가 부건의 형을 죽였기 때문에 350년에 부홍이 죽자 부건이 뒤를 이었다. 부건은 관중(關中)을 지배하던 두홍(杜洪)을 방심시키기 위해 삼진왕(三秦王)의 칭호를 버리고 동진(東晉)의 관작을 받았으며 주둔지에서 보리를 파종하였다. 두홍은 이러한 부건의 책략에 속아 방비를 하지 않았으며, 부건은 이를 틈타 장안(長安)을 점령하였다. 351년 정월에 부건은 진(秦)을 건국하고 천왕(天王)에 즉위하였으며, 352년에는 황제에 즉위

## HybridRetrieval(Dense_retrieval + Sparse_retrieval) 예제

In [13]:
# Dense_retrieval을 학습시켰던 tokenizer와 p_encdoer, q_encoder의 path를 인자로 받습니다.
hybrid_retrieval = HybridRetrieval(tokenizer, 'p_encoder/','q_encoder/')

/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [14]:
doc_ids, scores = hybrid_retrieval.get_topk_doc_id_and_score(querys[0],10)

/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [15]:
print(f'question : {querys[0]}')
for i in range(0,3):
    print(f'top_{i+1} passage score : {scores[i]}')
    print(hybrid_retrieval.wiki_id_context_dict[doc_ids[i]])
    print('-'*100)

question : 유령'은 어느 행성에서 지구로 왔는가?
top_1 passage score : 330.1960886796875
주인공인 비론 파릴(Biron Farrill)은 네페로스 행성의 지도자의 아들로서 지구의 대학에 유학을 왔다. 어느날 동급생 샌더 존티어(Sander Jonti)는 네페로스 행성이 속한 말머리 성운을 지배하는 타이란 제국에 의해 비론의 아버지가 체포를 당했다고 말하며 다른 별인 로디아(Rhodia)로 도망치기를 권한다. 그곳에 가서도 어려움을 겪다가 총독의 딸인 아르타(Artemisia) 그녀의 삼촌 질브레트(Gillbret)와 함께 우주선을 탈취해고 린겐(Lingane)별을 찾아간다. 거기서 아우타치(Autarch)(=샌더 존티어)의 정체를 알게되고 질브레트(Gillbret)가 말하는 숨겨진 혁명세력을 찾으러 같이 말머리 성운 안을 탐험하러 간다. 그곳의 한 행성에서 아우타치는 자신이 밀고자임을 말하며 비론을 죽이려고한다. 그러나 그가 배신자임을 알게된 측근에 의해 아우타치는 죽게되고 일행은 타이란 제국측에 체포된다. 질브레트는 결국 숨겨진 혁명 세력이란 자신의 공상임을 밝히고 처형 직전에 사면을 받아 일행은 목숨을 건지게된다. 그러나 혁명세력은 존재하였고 중심은 바로 로디아(Rhodia)였다고 로디아의 힌리크 총독이 밝히며 마지막에 이상적인 정치를 위한 고대문서로 '미국 헌법'이 언급한다.
----------------------------------------------------------------------------------------------------
top_2 passage score : 329.7473451914062
더크 젠틀리의 성스러운 탐정사무소의 줄거리는 이야기의 중추적인 부위에 자리잡은 시간 여행이란 주제 때문에 줄거리가 이어져 있지 않다. 40억년전 지구에서, 사락사라의 사람들이 자신들만의 낙원을 만들기 위해 지구에 착륙하였다. 그러나, 기술자의 게으름으로 인해서 착륙선은 폭발하였으며, 안에 타고 

In [16]:
q_ids, q_scores = hybrid_retrieval.get_topk_doc_id_and_score_for_querys(querys.to_list(),10)

print(type(q_ids))
print(type(q_scores))
print(q_ids[querys[0]])
print(q_scores[querys[0]])

100%|██████████| 600/600 [00:36<00:00, 16.34it/s]

<class 'dict'>
<class 'dict'>
[42242, 24024, 9781, 43280, 19993, 35064, 10509, 47081, 29886, 34043]
[330.1960886796875, 329.7473451914062, 329.4960632285156, 329.0064110390625, 327.820629109375, 324.9554672832031, 322.9897210058594, 322.6204131152344, 319.1797901699219, 317.1388856933594]


In [17]:
print(f'question : {querys[1]}')
for i in range(0,3):
    print(f'top_{i+1} passage score : {q_scores[querys[1]][i]}')
    print(hybrid_retrieval.wiki_id_context_dict[q_ids[querys[1]][i]])
    print('-'*100)

question : 용병회사의 경기가 좋아진 것은 무엇이 끝난 이후부터인가?
top_1 passage score : 344.11435286523437
타이의 고대무술 무어이보란이 현대화된 무에타이는 크게 람무아이와 크라비크라봉으로 나뉜다. 람무아이는 맨손 격투술이며, 크라비크라봉은 '크라비' 등의 무기를 사용한 기술이다. 그 중 한국에서 보편적인 것은 람무아이로, 단단한 신체 부위를 사용하여 상대방을 때려 부수는 격투 방식이다. 뼈를 더욱 견고하게 하고 파괴력을 높이기 위해 선수들은 정강이, 팔꿈치, 무릎 등을 혹독하게 단련하는데, 이 지나친 신체 혹사는 대부분의 무에타이 선수들이 조기에 은퇴하는 요인 중 하나다. 무에타이 선수는 경기 전에 전통 음악에 맞춰 경기장 주위를 돌며 복을 비는 뜻으로 와이크루라는 춤을 추며 경기를 준비한다. 복장은 경기에서 쓰는 트렁크 바지를 기본으로 하고, 남자는 기본적으로 웃통을 벗은 채로 한다. 장갑은 과거엔 손가락에 대마로 붕대를 감아 유리가루를 묻혀 사용했으나 위험하다 하여 현대에 와서는 좀더 완화된 경기용 글러브를 사용한다. 경기는 기본적으로 3분 5라운드로 진행하며 휴식 2분을 준다. 기본적으로 모든 기술들을 허용하나, 박치기, 물어뜯기, 던지기, 걸어 넘어뜨리기, 급소 공격은 경기에선 금지한다.
----------------------------------------------------------------------------------------------------
top_2 passage score : 342.6269297734375
냉전 종식 이후 전 세계적으로 소규모의 끊임없는 국지 분쟁들이 생겨나고 강대국들의 잦은 내정간섭과 문화침공, 강제간섭 등은 안정되지 않은 많은 나라의 골칫거리들이 되었다. 이처럼 많은 분쟁과 폭동, 혼란을 틈타 용병산업이 호황을 누리게 되었다. 2010년 용병회사들은 다국적 거대기업으로 성장하면서 각국의 정규군에 못지않은 화력을 갖추고 고도로 훈련된 외인부대들이 적게는 수십 명에